In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(1216)

# Loading the dataset
train_X = pd.read_csv("train_features.csv")
train_y = pd.read_csv("train_labels.csv")

merged_df = pd.merge(
    train_X,
    train_y, 
    on=['uid'],
    how='inner'
)

# Splitting the features to groups

There are many different features in this dataset, and it would be a bit overwhelming to immediately train a model on all available features. In addition, for explainability purposes, it is useful to have high-level semantic groups of features and use these groups for analytics.

Below is one of the possible groupings I used. Obviusoly, this is only one option and the features can be grouped in many other ways:

### Demographic Information
- **`age_03` / `age_12`**: Binned age group.
- **`urban_03` / `urban_12`**: Locality size. Either 0. <100,000 (rural) or 1. 100,000+ (urban).
- **`married_03` / `married_12`**: Marital status.
- **`n_mar_03` / `n_mar_12`**: Number of marriages.
- **`ragender`**: Gender.
- **`sgender_03` / `sgender_12`**: Spouse's gender.

### Socioeconomic Status
- **`edu_gru_03` / `edu_gru_12`**: Binned education level.
- **`employment_03` / `employment_12`**: Employment status.
- **`rearnings_03` / `rearnings_12`**: Earnings from employment.
- **`searnings_03` / `searnings_12`**: Spouse's earnings from employment.
- **`hincome_03` / `hincome_12`**: Household income.
- **`hinc_business_03` / `hinc_business_12`**: Household income from business.
- **`hinc_rent_03` / `hinc_rent_12`**: Household income from rent.
- **`hinc_assets_03` / `hinc_assets_12`**: Household income from financial assets.
- **`hinc_cap_03` / `hinc_cap_12`**: Household capital income.
- **`rinc_pension_03` / `rinc_pension_12`**: Income from pensions.
- **`sinc_pension_03` / `sinc_pension_12`**: Spouse's income from pensions.

### Health and Physical Limitations
- **`glob_hlth_03` / `glob_hlth_12`**: Self-reported global health.
- **`adl_dress_03` / `adl_dress_12`**: Has difficulty getting dressed.
- **`adl_walk_03` / `adl_walk_12`**: Has difficulty walking from one side of the room to the other.
- **`adl_bath_03` / `adl_bath_12`**: Has difficulty bathing themselves in a tub or shower.
- **`adl_eat_03` / `adl_eat_12`**: Has difficulty eating.
- **`adl_bed_03` / `adl_bed_12`**: Has difficulty getting in and out of bed.
- **`adl_toilet_03` / `adl_toilet_12`**: Has difficulty using the toilet.
- **`n_adl_03` / `n_adl_12`**: Number of activities of daily living (ADL) limitations (0-5).
- **`iadl_money_03` / `iadl_money_12`**: Has difficulty managing money.
- **`iadl_meds_03` / `iadl_meds_12`**: Has difficulty taking medications.
- **`iadl_shop_03` / `iadl_shop_12`**: Has difficulty shopping for groceries.
- **`iadl_meals_03` / `iadl_meals_12`**: Has difficulty preparing a hot meal.
- **`n_iadl_03` / `n_iadl_12`**: Number of instrumental activities of daily living (IADL) limitations (0-4).
- **`hypertension_03` / `hypertension_12`**: Has been diagnosed with hypertension.
- **`diabetes_03` / `diabetes_12`**: Has been diagnosed with diabetes.
- **`resp_ill_03` / `resp_ill_12`**: Has been diagnosed with respiratory illness.
- **`arthritis_03` / `arthritis_12`**: Has been diagnosed with arthritis/rheumatism.
- **`hrt_attack_03` / `hrt_attack_12`**: Has been told they had a heart attack.
- **`stroke_03` / `stroke_12`**: Has been told they had a stroke.
- **`cancer_03` / `cancer_12`**: Has been diagnosed with cancer.
- **`n_illnesses_03` / `n_illnesses_12`**: Number of illnesses (0-7).
- **`bmi_03` / `bmi_12`**: Binned body mass index.

### Mental Health and Well-being
- **`depressed_03` / `depressed_12`**: Most of the past week, felt depressed.
- **`hard_03` / `hard_12`**: Most of the past week, felt that everything was an effort.
- **`restless_03` / `restless_12`**: Most of the past week, felt that their sleep was restless.
- **`happy_03` / `happy_12`**: Most of the past week, felt happy.
- **`lonely_03` / `lonely_12`**: Most of the past week, felt lonely.
- **`enjoy_03` / `enjoy_12`**: Most of the past week, felt that they enjoyed life.
- **`sad_03` / `sad_12`**: Most of the past week, felt sad.
- **`tired_03` / `tired_12`**: Most of the past week, felt tired.
- **`energetic_03` / `energetic_12`**: Most of the past week, felt they had a lot of energy.
- **`n_depr_03` / `n_depr_12`**: Number of CES-D depressive symptoms (0-9).
- **`cesd_depressed_03` / `cesd_depressed_12`**: Has 5+ CES-D depressive symptoms.

### Lifestyle and Behavior
- **`exer_3xwk_03` / `exer_3xwk_12`**: Exercises 3+ times per week.
- **`alcohol_03` / `alcohol_12`**: Currently drinks alcohol.
- **`tobacco_03` / `tobacco_12`**: Currently smokes tobacco.
- **`test_chol_03` / `test_chol_12`**: Has had a cholesterol blood test.
- **`test_tuber_03` / `test_tuber_12`**: Has been tested for tuberculosis.
- **`test_diab_03` / `test_diab_12`**: Has been tested for diabetes.
- **`test_pres_03` / `test_pres_12`**: Has been tested for high blood pressure.
- **`hosp_03` / `hosp_12`**: Has been hospitalized at least one night in the last year.
- **`visit_med_03` / `visit_med_12`**: Has visited a doctor at least once in the last year.
- **`out_proc_03` / `out_proc_12`**: Has had at least one outpatient procedure in the last year.
- **`visit_dental_03` / `visit_dental_12`**: Has visited a dentist at least once in the last year.

### Social and Family Dynamics
- **`n_living_child_03` / `n_living_child_12`**: Binned number of living children.
- **`migration_03` / `migration_12`**: Has lived or worked in the U.S.
- **`decis_famil_03` / `decis_famil_12`**: Weight in family decisions.
- **`decis_personal_03` / `decis_personal_12`**: Weight over personal decisions.
- **`care_adult_12`**: Uses time to look after a sick or disabled adult.
- **`care_child_12`**: Uses time to look after children under 12.
- **`volunteer_12`**: Uses time to volunteer for a non-profit.
- **`attends_class_12`**: Uses time to attend training course, lecture, or class.
- **`attends_club_12`**: Uses time to attend sports or social club.
- **`reads_12`**: Uses time to read books, magazines, newspapers.
- **`games_12`**: Uses time to do crosswords, jigsaw puzzles, number games.
- **`table_games_12`**: Uses time to play tabletop games. E.g., cards, dominoes, chess.
- **`comms_tel_comp_12`**: Uses time to talk on the phone or send message/use the web on a computer.
- **`act_mant_12`**: Uses time to maintain a house, do repairs, garden, etc.
- **`tv_12`**: Uses time to watch television.
- **`sewing_12`**: Uses time to sew, embroider, knit, make crafts.
- **`satis_ideal_12`**: How much they agree with the statement that their life is close to ideal.
- **`satis_excel_12`**: How much they agree with the statement that life is excellent.
- **`satis_fine_12`**: How much they agree with the statement that they are satisfied with their life.
- **`cosas_imp_12`**: How much they agree with the statement that they have achieved the things in life that are important to them.
- **`wouldnt_change_12`**: How much they agree with the statement that they would change almost nothing about their life.
- **`memory_12`**: Self-reported memory.
- **`rrelgimp_03` / `rrelgimp_12`**: Importance of religion.
- **`rrfcntx_m_12`**: How often they see friends and relatives.
- **`rsocact_m_12`**: How often they have social activities.
- **`rrelgwk_12`**: Participates in weekly religious services.

### Health Insurance and Coverage
- **`imss_03` / `imss_12`**: Has health coverage with IMSS.
- **`issste_03` / `issste_12`**: Has health coverage with ISSSTE/ISSSTE Estatal.
- **`pem_def_mar_03` / `pem_def_mar_12`**: Has health coverage with PEMEX, Defensa, or Marina.
- **`insur_private_03` / `insur_private_12`**: Has health coverage with private health insurance.
- **`insur_other_03` / `insur_other_12`**: Has health coverage with other health insurance.
- **`seg_pop_12`**: Has health coverage with Seguro Popular.
- **`insured_03` / `insured_12`**: Has health insurance.

### Migration and U.S. Experience
- **`a16a_12`**: Year when respondent first left for the U.S., if they ever lived in the U.S.
- **`a21_12`**: Total years lived or worked in the U.S.
- **`a22_12`**: Main job type during longest stay in the U.S.
- **`a33b_12`**: U.S. residency status.
- **`a34_12`**: Speaks English.

### Housing and Environment
- **`j11_12`**: Floor material of residence.

### Parental Education
- **`rameduc_m`**: Mother's education level.
- **`rafeduc_m`**: Father's education level.

### Employment and Job History
- **`rjob_hrswk_03` / `rjob_hrswk_12`**: Hours per week that they worked at their main job.
- **`rjlocc_m_03` / `rjlocc_m_12`**: Category of their longest occupation.
- **`rjob_end_03` / `rjob_end_12`**: Year that their last job ended.
- **`rjobend_reason_03` / `rjobend_reason_12`**: Reason that their last job ended.


In [2]:
feature_groups = {
    "Demographic Information": [
        "age_03", "age_12", "urban_03", "urban_12", "married_03", "married_12",
        "n_mar_03", "n_mar_12", "ragender", "sgender_03", "sgender_12"
    ],
    "Socioeconomic Status": [
        "edu_gru_03", "edu_gru_12", "employment_03", "employment_12", "rearnings_03",
        "rearnings_12", "searnings_03", "searnings_12", "hincome_03", "hincome_12",
        "hinc_business_03", "hinc_business_12", "hinc_rent_03", "hinc_rent_12",
        "hinc_assets_03", "hinc_assets_12", "hinc_cap_03", "hinc_cap_12",
        "rinc_pension_03", "rinc_pension_12", "sinc_pension_03", "sinc_pension_12"
    ],
    "Health and Physical Limitations": [
        "glob_hlth_03", "glob_hlth_12", "adl_dress_03", "adl_dress_12", "adl_walk_03",
        "adl_walk_12", "adl_bath_03", "adl_bath_12", "adl_eat_03", "adl_eat_12",
        "adl_bed_03", "adl_bed_12", "adl_toilet_03", "adl_toilet_12", "n_adl_03",
        "n_adl_12", "iadl_money_03", "iadl_money_12", "iadl_meds_03", "iadl_meds_12",
        "iadl_shop_03", "iadl_shop_12", "iadl_meals_03", "iadl_meals_12", "n_iadl_03",
        "n_iadl_12", "hypertension_03", "hypertension_12", "diabetes_03", "diabetes_12",
        "resp_ill_03", "resp_ill_12", "arthritis_03", "arthritis_12", "hrt_attack_03",
        "hrt_attack_12", "stroke_03", "stroke_12", "cancer_03", "cancer_12",
        "n_illnesses_03", "n_illnesses_12", "bmi_03", "bmi_12"
    ],
    "Mental Health and Well-being": [
        "depressed_03", "depressed_12", "hard_03", "hard_12", "restless_03",
        "restless_12", "happy_03", "happy_12", "lonely_03", "lonely_12", "enjoy_03",
        "enjoy_12", "sad_03", "sad_12", "tired_03", "tired_12", "energetic_03",
        "energetic_12", "n_depr_03", "n_depr_12", "cesd_depressed_03", "cesd_depressed_12"
    ],
    "Lifestyle and Behavior": [
        "exer_3xwk_03", "exer_3xwk_12", "alcohol_03", "alcohol_12", "tobacco_03",
        "tobacco_12", "test_chol_03", "test_chol_12", "test_tuber_03", "test_tuber_12",
        "test_diab_03", "test_diab_12", "test_pres_03", "test_pres_12", "hosp_03",
        "hosp_12", "visit_med_03", "visit_med_12", "out_proc_03", "out_proc_12",
        "visit_dental_03", "visit_dental_12"
    ],
    "Social and Family Dynamics": [
        "n_living_child_03", "n_living_child_12", "migration_03", "migration_12",
        "decis_famil_03", "decis_famil_12", "decis_personal_03", "decis_personal_12",
        "care_adult_12", "care_child_12", "volunteer_12", "attends_class_12",
        "attends_club_12", "reads_12", "games_12", "table_games_12", "comms_tel_comp_12",
        "act_mant_12", "tv_12", "sewing_12", "satis_ideal_12", "satis_excel_12",
        "satis_fine_12", "cosas_imp_12", "wouldnt_change_12", "memory_12", "rrelgimp_03",
        "rrelgimp_12", "rrfcntx_m_12", "rsocact_m_12", "rrelgwk_12"
    ],
    "Health Insurance and Coverage": [
        "imss_03", "imss_12", "issste_03", "issste_12", "pem_def_mar_03", "pem_def_mar_12",
        "insur_private_03", "insur_private_12", "insur_other_03", "insur_other_12",
        "seg_pop_12", "insured_03", "insured_12"
    ],
    "Migration and U.S. Experience": [
        "a16a_12", "a21_12", "a22_12", "a33b_12", "a34_12"
    ],
    "Housing and Environment": [
        "j11_12"
    ],
    "Parental Education": [
        "rameduc_m", "rafeduc_m"
    ],
    "Employment and Job History": [
        "rjob_hrswk_03", "rjob_hrswk_12", "rjlocc_m_03", "rjlocc_m_12", "rjob_end_03",
        "rjob_end_12", "rjobend_reason_03", "rjobend_reason_12"
    ],
    "Identifier": [
        "uid", "year"
    ]
}


# Train / Validation split

**It is important to consider several things when approaching the train/validation split:**

### Temporal Structure:
 + Features are from 2003 and 2012
 + Target values (composite_scores) are from 2016 and 2021
 + This means we're always predicting 4-9 years into the future
 + Some individuals have scores for both 2016 and 2021, while others have only one score
Hence, we should try to maintain similar distributions of 2016/2021 predictions in both sets

### Correct 'out-of-sample' Predictions:
 + we must ensure no uid appears in both training and validation sets (this would be a form of data leakage in favor of the model)
 
### Best Practices
 + We should try to maintain similar distributions of 2016/2021 predictions in both sets
 + The validation set should mirror the test set structure to properly evaluate model performance

In [3]:
# Calculate the distribution of years in the target
year_dist = train_y['year'].value_counts(normalize=True)
print("Distribution of years:", year_dist)

# Get unique UIDs
unique_uids = train_y['uid'].unique()

# Stratify based on whether a person has one or both years
uid_patterns = train_y.groupby('uid')['year'].nunique()
single_year_uids = uid_patterns[uid_patterns == 1].index
both_year_uids = uid_patterns[uid_patterns == 2].index

train_uids_single, val_uids_single = train_test_split(
    single_year_uids, 
    test_size=0.2, 
    random_state=42
)

train_uids_both, val_uids_both = train_test_split(
    both_year_uids, 
    test_size=0.2, # 80 / 20 split
    random_state=42
)

# Combine the UIDs
train_uids = np.concatenate([train_uids_single, train_uids_both])
val_uids = np.concatenate([val_uids_single, val_uids_both])

# Split the actual data
train_indices = train_y['uid'].isin(train_uids)
val_indices = train_y['uid'].isin(val_uids)

y_train = train_y[train_indices]
y_val = train_y[val_indices]
X_train = merged_df[merged_df['uid'].isin(train_uids)]
X_val = merged_df[merged_df['uid'].isin(val_uids)]

Distribution of years: year
2021    0.632512
2016    0.367488
Name: proportion, dtype: float64


# Missing values

There are several features with high percentage of missing values. We will drop them for brevity of this notebook.

In [4]:
# dropping features that contain >70% missing values
features_to_drop = {
    "due_to_missing_values": ['a16a_12', 
                              'a22_12',                                   
                              'a21_12',                                     
                              'a33b_12',                                  
                              'rjlocc_m_03',                              
                              'rjob_end_03',                              
                              'rjobend_reason_03',                        
                              'rjob_end_12',
                              'rjobend_reason_12']
}
X_train = X_train.drop(columns=features_to_drop["due_to_missing_values"])
X_val = X_val.drop(columns=features_to_drop["due_to_missing_values"])


# Evaluation Method

We use a very empirical approach here - checking the predictive power of each group based on model performance. Since feature importance varies based on the model/prediction method, we will use both a tree-based model and statistical based methods to check the mean predictive power for each group.  
This would typically give us a good estimate of the non-linear predictive power of each group (tree-based) and the linear predictive power of each group (statistical).

**For each feature group**, we will:
 + Train a simple model and get feature importances
 + Evaluate model scores on validation set

In [5]:
def train_linear_model(X_train, y_train, random_state=42):
    """
    Train a linear model with preprocessing pipeline
    """
    # Create preprocessing pipeline
    num_imputer = SimpleImputer(strategy='mean')
    cat_imputer = SimpleImputer(strategy='constant', fill_value='MISSING')
    scaler = StandardScaler()
    
    # Separate numerical and categorical columns
    num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
    cat_cols = X_train.select_dtypes(include=['object']).columns
    
    # Initialize X_processed as None
    X_processed = None
    
    # Process numerical features if they exist
    if len(num_cols) > 0:
        X_num = X_train[num_cols].copy()
        X_num = num_imputer.fit_transform(X_num)
        X_num = scaler.fit_transform(X_num)
        X_num = pd.DataFrame(X_num, columns=num_cols, index=X_train.index)
        X_processed = X_num
    
    # Process categorical features if they exist
    if len(cat_cols) > 0:
        X_cat = X_train[cat_cols].copy()
        X_cat = cat_imputer.fit_transform(X_cat)
        X_cat = pd.DataFrame(X_cat, columns=cat_cols, index=X_train.index)
        X_cat = pd.get_dummies(X_cat, drop_first=True)
        X_processed = X_cat if X_processed is None else pd.concat([X_processed, X_cat], axis=1)
    
    # Train model
    model = Ridge(random_state=random_state)
    model.fit(X_processed, y_train)
    
    return {
        'model': model,
        'num_imputer': num_imputer if len(num_cols) > 0 else None,
        'cat_imputer': cat_imputer if len(cat_cols) > 0 else None,
        'scaler': scaler if len(num_cols) > 0 else None,
        'num_cols': num_cols,
        'cat_cols': cat_cols
    }

def train_nonlinear_model(X_train, y_train, random_state=42):
    """
    Train a non-linear model (Random Forest) with preprocessing pipeline
    """
    # Create preprocessing pipeline
    num_imputer = SimpleImputer(strategy='mean')
    cat_imputer = SimpleImputer(strategy='constant', fill_value='MISSING')
    
    # Separate numerical and categorical columns
    num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
    cat_cols = X_train.select_dtypes(include=['object']).columns
    
    # Initialize X_processed as None
    X_processed = None
    
    # Process numerical features if they exist
    if len(num_cols) > 0:
        X_num = X_train[num_cols].copy()
        X_num = num_imputer.fit_transform(X_num)
        X_num = pd.DataFrame(X_num, columns=num_cols, index=X_train.index)
        X_processed = X_num
    
    # Process categorical features if they exist
    if len(cat_cols) > 0:
        X_cat = X_train[cat_cols].copy()
        X_cat = cat_imputer.fit_transform(X_cat)
        X_cat = pd.DataFrame(X_cat, columns=cat_cols, index=X_train.index)
        X_cat = pd.get_dummies(X_cat, drop_first=True)
        X_processed = X_cat if X_processed is None else pd.concat([X_processed, X_cat], axis=1)
    
    # Train model
    model = RandomForestRegressor(n_estimators=100, random_state=random_state)
    model.fit(X_processed, y_train)
    
    return {
        'model': model,
        'num_imputer': num_imputer if len(num_cols) > 0 else None,
        'cat_imputer': cat_imputer if len(cat_cols) > 0 else None,
        'num_cols': num_cols,
        'cat_cols': cat_cols
    }

def evaluate_model(model_dict, X_eval, y_eval):
    """
    Evaluate model performance on evaluation set
    """
    # Apply same preprocessing as training
    num_cols = model_dict['num_cols']
    cat_cols = model_dict['cat_cols']
    
    # Initialize X_processed as None
    X_processed = None
    
    # Process numerical features if they exist
    if len(num_cols) > 0 and model_dict['num_imputer'] is not None:
        X_num = X_eval[num_cols].copy()
        X_num = model_dict['num_imputer'].transform(X_num)
        # Only scale if it's a linear model (which has a scaler)
        if 'scaler' in model_dict and model_dict['scaler'] is not None:
            X_num = model_dict['scaler'].transform(X_num)
        X_num = pd.DataFrame(X_num, columns=num_cols, index=X_eval.index)
        X_processed = X_num
    
    # Process categorical features if they exist
    if len(cat_cols) > 0 and model_dict['cat_imputer'] is not None:
        X_cat = X_eval[cat_cols].copy()
        X_cat = model_dict['cat_imputer'].transform(X_cat)
        X_cat = pd.DataFrame(X_cat, columns=cat_cols, index=X_eval.index)
        
        # Create dummy variables with the same columns as training
        feature_names = model_dict['model'].feature_names_in_
        X_cat_dummy = pd.get_dummies(X_cat, drop_first=True)
        
        # Add missing columns with zeros
        missing_cols = set(feature_names) - set(X_cat_dummy.columns)
        if len(num_cols) > 0:
            missing_cols = missing_cols - set(num_cols)
        
        for col in missing_cols:
            X_cat_dummy[col] = 0
            
        # Ensure columns are in the same order as during training
        cat_feature_names = [col for col in feature_names if col not in num_cols]
        X_cat = X_cat_dummy[cat_feature_names]
        
        X_processed = X_cat if X_processed is None else pd.concat([X_processed, X_cat], axis=1)
    
    # Ensure columns match training data
    X_processed = X_processed[model_dict['model'].feature_names_in_]
    
    # Make predictions
    y_pred = model_dict['model'].predict(X_processed)
    
    # Calculate metrics
    results = {
        'rmse': np.sqrt(mean_squared_error(y_eval, y_pred)),
        'mae': mean_absolute_error(y_eval, y_pred),
        'r2': r2_score(y_eval, y_pred)
    }
    
    return results

def plot_feature_groups_treemap(results_df):
    """
    Create a treemap visualization showing the predictive power of different feature groups.
    
    Args:
        results_df (pd.DataFrame): DataFrame containing model results with columns:
            group_name, n_features, linear_r2, nonlinear_r2, mean_predictive_power, mean_rmse
    """
    # Create a treemap visualization
    fig = px.treemap(
        results_df,
        path=[px.Constant("All Groups"), 'group_name'],
        values='n_features',
        color='mean_predictive_power',
        custom_data=['linear_r2', 'nonlinear_r2', 'mean_rmse', 'n_features'],
        color_continuous_scale='RdYlBu',
        color_continuous_midpoint=results_df['mean_predictive_power'].mean()
    )

    # Customize hover template
    fig.update_traces(
        hovertemplate="""
        <b>%{label}</b><br>
        Mean R² Score: %{color:.3f}<br>
        Linear R²: %{customdata[0]:.3f}<br>
        Non-linear R²: %{customdata[1]:.3f}<br>
        Mean RMSE: %{customdata[2]:.1f}<br>
        Number of features: %{customdata[3]}<br>
        """,
    )

    # Update layout
    fig.update_layout(
        title='Feature Groups Predictive Power',
        width=1000,
        height=600,
        coloraxis_colorbar_title='Mean R² Score'
    )

    fig.show()

    # Also print sorted results for reference
    print("\nGroups sorted by predictive power:")
    print(results_df.sort_values('mean_predictive_power', ascending=False)[
        ['group_name', 'mean_predictive_power', 'mean_rmse', 'n_features']
    ].to_string())

# Predictive Power Analysis

We apply the training and evaluation pipeline to each feature group, and compare the predictive power of linear and non-linear models.
The will be stored in the `results` DataFrame, and visualized as a treemap.


In [6]:

results = []
for group_name, features in feature_groups.items():
    if group_name != 'Identifier':  # Skip identifier group
        print(f"\nAnalyzing {group_name} features...")

        # remove features that are due to missing values
        features = [feature for feature in features if feature not in features_to_drop["due_to_missing_values"]]
        # Train models
        linear_model = train_linear_model(X_train[features], y_train['composite_score'])
        nonlinear_model = train_nonlinear_model(X_train[features], y_train['composite_score'])
        
        # Evaluate models
        linear_scores = evaluate_model(linear_model, X_val[features], y_val['composite_score'])
        nonlinear_scores = evaluate_model(nonlinear_model, X_val[features], y_val['composite_score'])
        
        # Add mean predictive power column
        mean_predictive_power = (linear_scores['r2'] + nonlinear_scores['r2']) / 2
        mean_rmse = (linear_scores['rmse'] + nonlinear_scores['rmse']) / 2

        results.append({
            'group_name': group_name,
            'n_features': len(features),
            'linear_rmse': linear_scores['rmse'],
            'linear_r2': linear_scores['r2'],
            'nonlinear_rmse': nonlinear_scores['rmse'],
            'nonlinear_r2': nonlinear_scores['r2'],
            'mean_predictive_power': mean_predictive_power,
            'mean_rmse': mean_rmse
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df.sort_values(by='mean_predictive_power', ascending=False)
plot_feature_groups_treemap(results_df)


Analyzing Demographic Information features...

Analyzing Socioeconomic Status features...

Analyzing Health and Physical Limitations features...

Analyzing Mental Health and Well-being features...

Analyzing Lifestyle and Behavior features...

Analyzing Social and Family Dynamics features...

Analyzing Health Insurance and Coverage features...

Analyzing Migration and U.S. Experience features...

Analyzing Housing and Environment features...

Analyzing Parental Education features...

Analyzing Employment and Job History features...



Groups sorted by predictive power:
                         group_name  mean_predictive_power  mean_rmse  n_features
1              Socioeconomic Status               0.436028  46.746835          22
5        Social and Family Dynamics               0.364883  49.627642          31
0           Demographic Information               0.240855  54.257590          11
9                Parental Education               0.202550  55.619547           2
2   Health and Physical Limitations               0.132379  58.000225          44
6     Health Insurance and Coverage               0.128281  58.136017          13
8           Housing and Environment               0.096791  59.192906           1
10       Employment and Job History               0.096313  59.179325           3
7     Migration and U.S. Experience               0.072397  59.986939           1
4            Lifestyle and Behavior               0.046189  60.794617          22
3      Mental Health and Well-being               0.007692  62

## Top Predictive Groups:
+ **Socioeconomic Status** (R² ≈ 0.44) is the strongest predictor, suggesting that economic factors are highly correlated with cognitive decline

+ **Social and Family Dynamics** (R² ≈ 0.36) is the second strongest, indicating the importance of social connections and family support

+ **Demographic Information** (R² ≈ 0.24) shows moderate predictive power

## Middle Range:
+ **Parental Education** (R² ≈ 0.20) has decent predictive power despite having only 2 features

+ **Health and Physical Limitations** (R² ≈ 0.13) surprisingly shows relatively lower predictive power despite having 44 features

+ **Health Insurance and Coverage** (R² ≈ 0.13) also shows moderate predictive power

## Weaker Predictors:
+   **Housing and Environment**, **Employment and Job History**, and **Migration and U.S. Experience** all show relatively low predictive power (R² < 0.10)

<br>

# Key Insights:
+ **Social determinants** (economic status and social connections) appear more predictive than direct health indicators

+ Some small feature groups (like **Parental Education**) punch above their weight in terms of predictive power

+ The number of features in a group doesn't necessarily correlate with its predictive power (e.g., **Health and Physical Limitations** has 44 features but relatively low R²)
